In [4]:
import socket
import threading
import json
from flask import Flask, render_template_string
from flask_socketio import SocketIO, emit

app = Flask(__name__)
socketio = SocketIO(app)

track_data = {}

# HTML-Template für die Anzeige der Track-Daten
html_template = """
<!doctype html>
<html lang="en">
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
    <title>Track Information</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.0.1/socket.io.js"></script>
    <script type="text/javascript" charset="utf-8">
        document.addEventListener("DOMContentLoaded", () => {
            var socket = io();
            socket.on('update', function(data) {
                document.getElementById('track_title').innerText = data.track_title;
                document.getElementById('track_artist').innerText = data.track_artist;
                document.getElementById('track_genre').innerText = data.track_genre;
                document.getElementById('track_key').innerText = data.track_key;
                document.getElementById('bpm').innerText = data.bpm;
                document.getElementById('phrase_type').innerText = data.phrase_type;
                document.getElementById('phrase_section').innerText = data.phrase_section;
                document.getElementById('master_player_number').innerText = data.master_player_number;
                document.getElementById('track_time_reached').innerText = data.track_time_reached;
                document.getElementById('track_comment').innerText = data.track_comment;
                document.getElementById('track_bank').innerText = data.track_bank;
                document.getElementById('fill').innerText = data.fill;
                document.getElementById('beat').innerText = data.beat;
                document.getElementById('track_label').innerText = data.track_label;
            });
        });
    </script>
</head>
<body>
    <div>
        <h1>Track Information</h1>
        <p><strong>Track Title:</strong> <span id="track_title"></span></p>
        <p><strong>Artist:</strong> <span id="track_artist"></span></p>
        <p><strong>Genre:</strong> <span id="track_genre"></span></p>
        <p><strong>Key:</strong> <span id="track_key"></span></p>
        <p><strong>BPM:</strong> <span id="bpm"></span></p>
        <p><strong>Phrase Type:</strong> <span id="phrase_type"></span></p>
        <p><strong>Phrase Section:</strong> <span id="phrase_section"></span></p>
        <p><strong>Master Player Number:</strong> <span id="master_player_number"></span></p>
        <p><strong>Time Reached:</strong> <span id="track_time_reached"></span></p>
        <p><strong>Comment:</strong> <span id="track_comment"></span></p>
        <p><strong>Track Bank:</strong> <span id="track_bank"></span></p>
        <p><strong>Fill:</strong> <span id="fill"></span></p>
        <p><strong>Beat:</strong> <span id="beat"></span></p>
        <p><strong>Label:</strong> <span id="track_label"></span></p>
    </div>
</body>
</html>
"""

@app.route('/')
def index():
    return render_template_string(html_template)

def update_track_info(json_data):
    global track_data
    track_data = {
        "phrase_type": json_data.get("phraseType"),
        "track_key": json_data.get("trackKey"),
        "track_comment": json_data.get("trackComment"),
        "master_player_number": json_data.get("masterPlayerNumber"),
        "track_time_reached": json_data.get("trackTimereached"),
        "phrase_section": json_data.get("phraseSection"),
        "track_artist": json_data.get("trackArtist"),
        "track_genre": json_data.get("trackGenre"),
        "bpm": json_data.get("bpm"),
        "track_bank": json_data.get("trackBank"),
        "fill": json_data.get("fill"),
        "beat": json_data.get("beat"),
        "track_title": json_data.get("trackTitle"),
        "track_label": json_data.get("trackLabel")
    }
    socketio.emit('update', track_data)

def socket_receiver():
    # Erstellen eines UDP-Sockets
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    port = 7000
    sock.bind(('', port))
    print("Socket verbunden mit Port", port)
    receiving = True

    try:
        while receiving:
            print('\nWarten auf eine Nachricht')
            data, address = sock.recvfrom(4096)
            print('Erhaltene {} Bytes von {}'.format(len(data), address))

            try:
                json_data = json.loads(data)  # Daten dekodieren
                print("JSON-Daten erfolgreich dekodiert:", json_data)
                update_track_info(json_data)
            except json.JSONDecodeError as e:
                print("Fehler beim Dekodieren der JSON-Daten:", e)
    finally:
        sock.close()  # Socket schließen

socketio.run(app, debug=True, use_reloader=False, allow_unsafe_werkzeug=True)


Werkzeug appears to be used in a production deployment. Consider switching to a production web server instead.


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [19/Jul/2024 17:53:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2024 17:53:50] "GET /socket.io/?EIO=4&transport=polling&t=P3Binc7 HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2024 17:53:50] "POST /socket.io/?EIO=4&transport=polling&t=P3BincG&sid=BuC5mM8k6B8fuW3UAAAA HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2024 17:53:50] "GET /socket.io/?EIO=4&transport=polling&t=P3BincG.0&sid=BuC5mM8k6B8fuW3UAAAA HTTP/1.1" 200 -
